# Ngrok을 통한 vllm Endpoint 제공 방법

## 개요

* colab 내에서 vllm을 통해 LLM 모델 실행 및 엔드포인트 실행
* 해당 엔드포인트를 외부에서 접근가능하도록 ngrok을 통해 외부 endpoint와 연결

## * ngrok 및 vllm 패키지 설치

In [ ]:
# 1. 기존 충돌 패키지 정리 및 필수 의존성 선제 설치
!pip install --upgrade pip

# 2. vLLM 최신 안정 버전 설치 (0.5.4 대신 최신 버전 권장)
# 최신 버전은 pyairports 등의 자질구레한 의존성 문제가 해결되었습니다.
!pip install vllm pyngrok

In [ ]:
# 설치후 세션 강제 재시작
import os
os.kill(os.getpid(), 9)

* 패키지 설치 후, 환경변수 설정 필요
  * NGROK_TOKEN
    * ngrok token을 키값으로 입력
    * https://dashboard.ngrok.com/get-started/your-authtoken

## * Ngrok Endpoint URL 생성

In [ ]:
import os
from pyngrok import ngrok
from google.colab import userdata

NGROK_TOKEN = userdata.get('NGROK_TOKEN')
os.environ["NGROK_TOKEN"] = NGROK_TOKEN

ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000).public_url
print(f"🚀 API 주소 생성 완료: {public_url}")
print(f"👉 이 주소를 복사해서 사용하세요: {public_url}/v1")

## * Cloudflare Tunnel 사용

In [ ]:
import subprocess
import time
import re
import os

# 1. cloudflared 바이너리 다운로드 (없을 경우)
if not os.path.exists("cloudflared-linux-amd64"):
    print("Downloading cloudflared...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64

# 2. 로그 파일 초기화
log_file = open("tunnel.log", "w")

# 3. 백그라운드에서 cloudflared 실행 (포트 8000 기준)
# vLLM이 기본적으로 8000 포트를 사용하므로 --url http://localhost:8000 설정
print("Starting Cloudflare Tunnel...")
process = subprocess.Popen(
    ['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8000'],
    stdout=log_file,
    stderr=log_file
)

# 4. URL이 생성될 때까지 로그 모니터링
url = None
max_retries = 20  # 최대 20초 대기
for _ in range(max_retries):
    time.sleep(1)
    with open("tunnel.log", "r") as f:
        content = f.read()
        # 정규표현식으로 trycloudflare.com URL 찾기
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
        if match:
            url = match.group(0)
            break

if url:
    print(f"\n✅ Tunnel Public URL: {url}")
    print("👉 이 URL을 API Client의 base_url로 사용하세요 (뒤에 /v1 붙이기)")
else:
    print("\n❌ URL 생성 실패. tunnel.log를 확인하세요.")
    # 실패 시 프로세스 종료
    process.kill()


## * vllm 서버 실행

In [ ]:
!python -c "import vllm; print(vllm.__version__)"

0.13.0


### * vLLM 실행 시 지정 가능한 주요 옵션

#### - 일반 옵션

|범주|옵션|기능 및 역할|
| ----- | ------------------------ | ----------------------------------------- |
|네트워크 및 연결|--host| 서버가 수신할 IP 주소 (기본값: localhost). 외부 접속 허용 시 0.0.0.0 사용|
||--port|API 서버 포트 (기본값: 8000).|
||--api-key|API 호출 시 사용할 인증 키 설정. <br> (설정 시 클라이언트 헤더에 Authorization: Bearer <KEY> 필수)|
||--ssl-keyfile|HTTPS 통신을 위한 SSL 인증서 키 파일 경로.|
||--ssl-certfile|HTTPS 통신을 위한 SSL 인증서 파일 경로.|
|모델 및 토크나이저 제어|--served-model-name|API 호출 시 사용할 모델의 별칭(Alias). 여러 개 지정 가능 <br> (예: --served-model-name gpt-3.5-turbo my-model).|
||--chat-template|채팅 템플릿(Jinja2) 파일 경로 지정. 기본 템플릿이 마음에 안 들거나 커스텀 형식이 필요할 때 유용.|
||--response-role|챗봇 응답의 역할 지정 (기본값: assistant).|
||--tokenizer-mode|토크나이저 로딩 방식 (auto, slow). 가끔 auto가 실패할 때 slow로 강제 지정.|
|로깅 및 모니터링|--disable-log-requests| API 요청(입출력 텍스트 포함) 로그를 끕니다. 보안이 중요하거나 로그가 너무 많을 때 사용.|
||--root-path|리버스 프록시(Nginx 등) 뒤에서 실행될 때 API의 루트 경로 지정 (예: /vllm).|


#### - 성능 관련 옵션

| 범주    | 옵션 (Argument)            | 기능 및 역할                                   | T4 환경 추천               |
| ----- | ------------------------ | ----------------------------------------- | ---------------------- |
| 추론 가속 | --speculative-config     | 다음 토큰을 미리 추측하여 생성 속도 향상 (EAGLE, N-gram 등) | N-gram (메모리 0 소모)      |
|       | --num-speculative-tokens | 한 번에 추측할 토큰 개수                            | 5 (안정적)                |
| 메모리   | --gpu-memory-utilization | vLLM이 예약할 GPU 메모리 비율 (0.0 ~ 1.0)          | 0.9 (남은 1.6GB는 OS/기타용) |
|       | --max-model-len          | 입력+출력 토큰의 최대 길이 제한                        | 4096 (뉴스 기사 대응)        |
| 처리량   | --enable-chunked-prefill | 긴 프롬프트를 쪼개서 처리하여 렉 방지 및 동시성 향상            | Enable (필수)            |
|       | --max-num-batched-tokens | 한 번의 연산(Iteration)에 처리할 최대 토큰 수           | 2048 (T4 성능 고려)        |
|       | --max-num-seqs           | 동시에 처리할 최대 요청(Request) 개수                 | 8 (메모리 한계 고려)          |
| 시스템   | --enforce-eager          | CUDA Graph 끄기 (디버깅용, 속도 느림)               | 미사용 (성능 위해 끔)          |
|       | --disable-log-stats      | 불필요한 로그 출력 방지                             | 사용                     |

<br>

In [ ]:
# 1. Normal
# 전통적인 LLM 추론 방식, 한 번에 하나의 토큰만 생성하고 검증

!python -m vllm.entrypoints.openai.api_server \
  --model Eslzzyl/Qwen3-4B-Instruct-2507-AWQ  \
  --max-model-len 2048 \
  --gpu-memory-utilization 0.85 \
  --disable-log-stats \
  --max-num-seqs 1

In [ ]:
# 2. EAGLE (Speculative Decoding with Draft Model)
# 별도의 Draft Model(AngelSlim/Qwen3-4B_eagle3)이 미래의 토큰 5개(num_speculative_tokens: 5)를 미리 '추측'하고, 메인 모델(AWQ)이 이를 한 번에 검증(Verify)하는 방식
# 메인 모델과 Draft 모델의 토크나이저/임베딩이 정확히 일치해야함
# TPS 상승이 가능하나, Draft 모델도 Vram에 올라가므로 VRAM 사용률 증가

!python -m vllm.entrypoints.openai.api_server \
  --model Eslzzyl/Qwen3-4B-Instruct-2507-AWQ \
  --max-model-len 2048 \
  --speculative-config '{"method": "eagle", "model": "AngelSlim/Qwen3-4B_eagle3", "num_speculative_tokens": 5}' \
  --gpu-memory-utilization 0.85 \
  --max-num-seqs 1 \
  --disable-log-stats

In [ ]:
# 3. N-gram (Prompt Lookup Decoding)
# 딥러닝 모델 대신, CPU 기반의 알고리즘이 현재 생성 중인 문맥이 프롬프트나 이전에 생성된 텍스트에 이미 존재하는지(N-gram 매칭) 확인하고, 존재한다면 그 뒤에 이어졌던 단어들을 그대로 가져와 제안하는 방식
# ngram_prompt_lookup_max: 4 -> 과거 4개 단어의 패턴이 반복되면 그 뒤를 예측
# 추가 모델 로드가 필요 없어 VRAM 절약 가능, 변수명이 반복되는 코드 생성이나, 본문 내용을 인용하는 요약 태스크에서 매우 높은 성능
# 창의적인 소설 쓰기나 대화처럼 반복 패턴이 없는 텍스트에서는 가속 효과가 거의 없음

!python -m vllm.entrypoints.openai.api_server \
  --model Eslzzyl/Qwen3-4B-Instruct-2507-AWQ  \
  --max-model-len 2048 \
  --speculative-config '{"method": "ngram", "ngram_prompt_lookup_max": 4, "ngram_prompt_lookup_min": 1, "num_speculative_tokens": 5}' \
  --gpu-memory-utilization 0.85 \
  --max-num-seqs 1 \
  --disable-log-stats

In [ ]:
# 최적화
# [가속] N-gram 방식 (메모리 추가 소모 없음)
# [최적화] 긴 뉴스 기사 처리를 위한 Chunked Prefill
# [기타] 로그 비활성화 및 포트 설정 (필요시 변경)
!python -m vllm.entrypoints.openai.api_server \
  --model Eslzzyl/Qwen3-4B-Instruct-2507-AWQ \
  --served-model-name "Qwen3-4B-Instruct" \
  --max-model-len 4096 \
  --gpu-memory-utilization 0.90 \
  --max-num-seqs 8 \
  --speculative-config '{"method": "ngram", "ngram_prompt_lookup_max": 4, "ngram_prompt_lookup_min": 1, "num_speculative_tokens": 5}' \
  --enable-chunked-prefill \
  --max-num-batched-tokens 2048 \
  --disable-log-stats \
  --disable-log-requests #API 요청 로그 출력 비활성화

## * Cloudflare Tunnel + vllm 백그라운드 실행

In [ ]:
import subprocess
import time
import re
import os
import requests

# ==========================================
# 1. Cloudflare Tunnel 설치 및 실행
# ==========================================
if not os.path.exists("cloudflared-linux-amd64"):
    print("📥 Downloading cloudflared...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64

# 터널 로그 파일
tunnel_log = open("tunnel.log", "w")

print("🚀 Starting Cloudflare Tunnel...")
tunnel_process = subprocess.Popen(
    ['./cloudflared-linux-amd64', 'tunnel', '--url', 'http://localhost:8000'],
    stdout=tunnel_log,
    stderr=tunnel_log
)

# URL 추출 대기
public_url = None
for _ in range(30):
    time.sleep(1)
    with open("tunnel.log", "r") as f:
        content = f.read()
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
        if match:
            public_url = match.group(0)
            break

if not public_url:
    print("❌ 터널 생성 실패. 로그를 확인하세요.")
    tunnel_process.kill()
else:
    print(f"✅ Tunnel URL: {public_url}")

# ==========================================
# 2. vLLM 서버 백그라운드 실행
# ==========================================
# vLLM 로그 파일
vllm_log = open("vllm.log", "w")

cmd = [
    "python", "-m", "vllm.entrypoints.openai.api_server",
    "--model", "Eslzzyl/Qwen3-4B-Instruct-2507-AWQ",
    "--served-model-name", "Qwen3-4B-Instruct",
    "--max-model-len", "4096",
    "--gpu-memory-utilization", "0.90",
    "--max-num-seqs", "8",
    "--port", "8000",
    "--speculative-config", '{"method": "ngram", "ngram_prompt_lookup_max": 4, "ngram_prompt_lookup_min": 1, "num_speculative_tokens": 5}',
    "--enable-chunked-prefill",
    "--max-num-batched-tokens", "2048",
    "--disable-log-stats",
    "--disable-log-requests"
]

print("\n🔥 Starting vLLM Server (Background)...")
vllm_process = subprocess.Popen(
    cmd,
    stdout=vllm_log,
    stderr=vllm_log
)

# ==========================================
# 3. 서버 준비 상태 체크 (Health Check)
# ==========================================
print("⏳ Waiting for model to load...", end="", flush=True)

start_time = time.time()
server_ready = False

while True:
    try:
        # 로컬 Health Check 엔드포인트 호출
        response = requests.get("http://localhost:8000/health")
        if response.status_code == 200:
            server_ready = True
            break
    except requests.exceptions.ConnectionError:
        pass

    # 프로세스가 죽었는지 확인
    if vllm_process.poll() is not None:
        print("\n❌ vLLM 프로세스가 종료되었습니다. vllm.log를 확인하세요.")
        break

    time.sleep(5)
    print(".", end="", flush=True)

if server_ready:
    load_time = time.time() - start_time
    print(f"\n\n🎉 Server Ready! ({load_time:.1f}s)")
    print(f"👉 API Base URL: {public_url}/v1")
    print(f"👉 Model Name: Qwen3-4B-Instruct")

    # ==========================================
    # 4. 무한 루프 (Colab 종료 방지)
    # ==========================================
    try:
        while True:
            time.sleep(60)
    except KeyboardInterrupt:
        print("\n🛑 Stopping servers...")
        vllm_process.terminate()
        tunnel_process.terminate()
else:
    print("\n❌ Server failed to start.")
    # 오류 내용 일부 출력
    with open("vllm.log", "r") as f:
        print(f.read()[-1000:]) # 마지막 1000자 출력
